In [1]:
from imdb import Cinemagoer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer # For stemming the sentence
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn import svm


import string
import contractions
import nltk, random, re
from sklearn.model_selection import train_test_split

#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('stopwords')
#nltk.download('vader_lexicon')




In [2]:
import numpy as np
import pickle
import sys
import re
sys.path.append("..")
from dataAggregation.json_processor import json_unzip


In [3]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
import re

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()


def preprocess(text, is_sw_remove=True):
    # Steps:
    # 1. lowercase
    # 2. Lammetize. (It does not stem. Try to preserve structure not to overwrap with potential acronym).
    # 3. Remove stop words.
    # 4. Remove punctuations.
    # 5. Remove character with the length size of 1.

    lowered = str.lower(text)
    lowered = re.sub(r"[^a-zA-Z0-9 ]", "", lowered)

    if is_sw_remove:
        stop_words = set(stopwords.words('english'))
    else:
        stop_words = set()

    word_tokens = word_tokenize(lowered)

    words = []
    for w in word_tokens:
        if w not in stop_words:
            if w not in string.punctuation:
                if len(w) > 1:
                    lemmatized = lemmatizer.lemmatize(w)
                    words.append(lemmatized)

    return words


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Justin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Load Compressed Dataset
zipped_dataset = pickle.load(open("../../data/movies_reviews_zip.p", "rb"))

# Unzip the Dataset
movie_data = json_unzip(zipped_dataset)

# Create Movie ID
movie_id_list = list(movie_data)

In [10]:
def lemmatize(text):
          wordnet_lemmatizer = WordNetLemmatizer()

In [8]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r'http\S+', '', phrase)
    return phrase

In [5]:
def get_sentiment_label(sentiment):
    if sentiment['compound'] >= 0.50:
        return 'pos'
    elif sentiment['compound'] <= -0.50:
        return 'neg'
    else:
        return 'neu'

def tokenize_reviews(reviews):
    stop_words = set(stopwords.words('english'))                #Generate list of stopwords
    results = []
    for i in range(len(reviews)):
        phrase = decontracted(reviews[i][0])            #Split contractions to original form (doesn't = does not)
        nltk_tokens = nltk.word_tokenize(phrase)                #Tokenize review for [i]
        new_list = [word.strip() for word in nltk_tokens]       #Strip leading and trailing whitespaces
        new_list = [new_list.lower() for new_list in new_list]  #Lowercase all words
        new_list = ["".join(j for j in i if j not in string.punctuation) for i in new_list] #Get rid of punctuation
        lemmatized_word = [lemmatize(word) for word in new_list]
        
        while("" in new_list):                                  #Remove leftover whitespace (list still keeps the "" after deconstructing the words)
            new_list.remove("")
        
        filtered_sentence = [w for w in new_list if not w in stop_words] #List comprehension for removing stopwords]
        
        sia = SentimentIntensityAnalyzer()
        polarity_scores = sia.polarity_scores(' '.join(filtered_sentence)) #Store sentiment scores
        # print(polarity_scores)
        sentiment = get_sentiment_label(polarity_scores)
        results.append((filtered_sentence, sentiment))
    return results
        
# documents = tokenize_reviews(reviews)

In [56]:
random.seed(1337)

#PROTOTYPE 
#The code below will fail if there are less than 2 sentiment values (most reviews (if not all) return neutral, and sometimes positive)
def analysis_function(documents):
    random.shuffle(documents)

    if (len(documents) == 1):
        return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0}
    
    train, test = train_test_split(documents, test_size = 0.33, random_state=42)

    x_train = [' '.join(words) for (words, label) in train]
    x_test = [' '.join(words) for (words, label) in test]
    y_train = [label for (words, label) in train]
    train_labels = set(y_train)
    if len(train_labels) <= 1:
        return {'neg': 0.0, 'neu': 0.0, 'pos': 0.0}
    
    y_test = [label for (words, label) in test]

    tfidf_vec = TfidfVectorizer(token_pattern = r'[a-zA-Z]+')
    x_train_bow = tfidf_vec.fit_transform(x_train) # fit train
    x_test_bow = tfidf_vec.transform(x_test) # transform test

    model_svm = svm.SVC(C=8.0, kernel='linear')
    model_svm.fit(x_train_bow, y_train)

    y_pred = model_svm.predict(x_test_bow)

    f1_scores = f1_score(y_test, y_pred, 
             average=None, 
             labels = ['neg', 'neu', 'pos'])
    return {
        'neg': f1_scores[0],
        'neu': f1_scores[1],
        'pos': f1_scores[2],
    }

1. tokenize
2. strip trailing whitespace
3. remove stopwords
4. bigrams/trigrams
5. sentiment analysis
  - for each review in a movie, store the pos, neutral, neg values
  - Plot the scores using 3 lines (one for each sentiment) against the review ID/number

In [48]:
import warnings
warnings.filterwarnings('ignore')

results = {}
for movie_id in movie_id_list:
    reviews = movie_data[movie_id]
    documents = tokenize_reviews(reviews)
    res = analysis_function(documents)
    results[movie_id] = res

ValueError: With n_samples=1, test_size=0.33 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [62]:
import json
with open('f1_scores.json', 'w') as fp:
    json.dump(results, fp, indent = 4)

In [14]:
def get_sentiments(reviews):
    stop_words = set(stopwords.words('english'))                #Generate list of stopwords
    results = {
        'neg': 0, 
        'neu': 0,
        'pos': 0,
    }
    for i in range(len(reviews)):
        phrase = decontracted(reviews[i][0])            #Split contractions to original form (doesn't = does not)
        nltk_tokens = nltk.word_tokenize(phrase)                #Tokenize review for [i]
        new_list = [word.strip() for word in nltk_tokens]       #Strip leading and trailing whitespaces
        new_list = [new_list.lower() for new_list in new_list]  #Lowercase all words
        new_list = ["".join(j for j in i if j not in string.punctuation) for i in new_list] #Get rid of punctuation
        lemmatized_word = [lemmatize(word) for word in new_list]
        
        while("" in new_list):                                  #Remove leftover whitespace (list still keeps the "" after deconstructing the words)
            new_list.remove("")
        
        filtered_sentence = [w for w in new_list if not w in stop_words] #List comprehension for removing stopwords]
        
        sia = SentimentIntensityAnalyzer()
        polarity_scores = sia.polarity_scores(' '.join(filtered_sentence)) #Store sentiment scores
        label = get_sentiment_label(polarity_scores)
        results[label] += 1  
    return results

In [15]:
import warnings
warnings.filterwarnings('ignore')

results = {}
for movie_id in movie_id_list:
    reviews = movie_data[movie_id]
    results[movie_id] = get_sentiments(reviews)

In [16]:
results

{'0064354': {'neg': 0, 'neu': 1, 'pos': 2},
 '0100680': {'neg': 0, 'neu': 3, 'pos': 17},
 '0047200': {'neg': 2, 'neu': 3, 'pos': 3},
 '0893406': {'neg': 4, 'neu': 3, 'pos': 10},
 '0277931': {'neg': 0, 'neu': 2, 'pos': 3},
 '0479948': {'neg': 6, 'neu': 0, 'pos': 7},
 '0101829': {'neg': 8, 'neu': 4, 'pos': 18},
 '0127388': {'neg': 2, 'neu': 0, 'pos': 2},
 '0080486': {'neg': 12, 'neu': 1, 'pos': 1},
 '0114366': {'neg': 0, 'neu': 1, 'pos': 3},
 '0117382': {'neg': 1, 'neu': 0, 'pos': 1},
 '0220631': {'neg': 7, 'neu': 2, 'pos': 2},
 '0087951': {'neg': 1, 'neu': 1, 'pos': 12},
 '0073766': {'neg': 0, 'neu': 1, 'pos': 23},
 '0031999': {'neg': 0, 'neu': 1, 'pos': 1},
 '0099300': {'neg': 1, 'neu': 0, 'pos': 6},
 '0213446': {'neg': 0, 'neu': 0, 'pos': 9},
 '0277941': {'neg': 2, 'neu': 4, 'pos': 24},
 '0182225': {'neg': 1, 'neu': 2, 'pos': 4},
 '0846119': {'neg': 0, 'neu': 0, 'pos': 4},
 '0760179': {'neg': 2, 'neu': 1, 'pos': 5},
 '0081421': {'neg': 2, 'neu': 0, 'pos': 0},
 '0785025': {'neg': 0, 'n

In [17]:
import json
with open('review_sentiment_counts.json', 'w') as fp:
    json.dump(results, fp, indent = 4)